In [13]:
import scipy.stats as stats
import pandas as pd
import psycopg2
import numpy as np
import math
import json
import os
import backtrader as bt
import datetime
import matplotlib

In [2]:
# Connect to db
conn = psycopg2.connect('dbname=securities user=postgres')
cur = conn.cursor()

In [5]:
query = '''
    SELECT *
    FROM prices
    ORDER BY symbol ASC, datetime ASC
'''
cur.execute(query)
data = cur.fetchall()

price_df = pd.DataFrame(data, columns = ['symbol', 'date', 'open', 'high', 'low', 'close'])

In [7]:
spy_price_df = price_df[price_df['symbol'] == 'SPY']

# Make sure backtrader can find column that is in datetime format
spy_price_df.loc[:,'date'] = pd.to_datetime(spy_price_df['date']) 
spy_price_df = spy_price_df.set_index('date')

/Users/vtamprateep/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
def momentum_algorithm(data):
    sma_30 = 

In [46]:
class SMAIndicator(bt.Indicator):
    lines = ('sma',)
    params = {
        'long_ma': None,
        'short_ma': None,
    }
    
    def __init__(self):
        self.addminperiod(max(self.params.long_ma, self.params.short_ma))
        self.long_period = max(self.params.long_ma, self.params.short_ma)
        self.short_period = min(self.params.long_ma, self.params.short_ma)
    
    def next(self):
        sma_long = np.mean(self.data.get(size = self.long_period))
        sma_short = np.mean(self.data.get(size = self.short_period))
        self.lines.sma[0] = sma_short - sma_long

In [69]:
class TestStrategy(bt.Strategy):
    
    params = {
        'trade_freq': None,
    }
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        self.sma = SMAIndicator(long_ma = 30, short_ma = 10)
        self.days = 0
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, PRICE: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            else:
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            self.bar_executed = len(self)
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log(
            'OPERATION PROFIT, GROSS %.2f, NET %.2f' % (
                trade.pnl,
                trade.pnlcomm,
            )
        )
        
    def next(self):
        self.days += 1
        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return
        
        if self.days > self.params.trade_freq - 1:
            self.days = 0
            if not self.position:
                if self.sma > 0:
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.order = self.buy()
            elif self.sma < 0:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [70]:
cerebro = bt.Cerebro()

cerebro_data = bt.feeds.PandasData(dataname = spy_price_df, datetime = -1)
cerebro.adddata(cerebro_data)
cerebro.addstrategy(TestStrategy, trade_freq = 10)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

In [71]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2019-10-09, Close, 291.27
2019-10-10, Close, 293.24
2019-10-11, Close, 296.28
2019-10-14, Close, 295.95
2019-10-15, Close, 298.88
2019-10-16, Close, 298.40
2019-10-17, Close, 299.28
2019-10-18, Close, 297.97
2019-10-21, Close, 299.99
2019-10-22, Close, 299.01
2019-10-22, BUY CREATE, 299.01
2019-10-23, BUY EXECUTED, PRICE: 298.73, Cost: 2987.30, Comm 0.00
2019-10-23, Close, 299.88
2019-10-24, Close, 300.37
2019-10-25, Close, 301.60
2019-10-28, Close, 303.30
2019-10-29, Close, 303.21
2019-10-30, Close, 304.14
2019-10-31, Close, 303.33
2019-11-01, Close, 306.14
2019-11-04, Close, 307.37
2019-11-05, Close, 307.03
2019-11-06, Close, 307.10
2019-11-07, Close, 308.18
2019-11-08, Close, 308.94
2019-11-11, Close, 308.35
2019-11-12, Close, 309.00
2019-11-13, Close, 309.10
2019-11-14, Close, 309.55
2019-11-15, Close, 311.79
2019-11-18, Close, 312.02
2019-11-19, Close, 311.93
2019-11-20, Close, 310.77
2019-11-21, Close, 310.27
2019-11-22, Close, 310.96
2019-11-25

In [56]:
cerebro.plot()

ValueError: Axis limits cannot be NaN or Inf